In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import re

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


In [3]:
df = pd.read_excel('ESTADIC//Base_ESTADIC_2019.xlsx', sheet_name='Segurança pública', nrows=29)
df

,CodUF,ESEG01,ESEG021,ESEG022,ESEG023,ESEG024,ESEG0241,ESEG03,ESEG04,ESEG05,...,ESEG5924,ESEG5925,ESEG60,ESEG611,ESEG612,ESEG613,ESEG614,ESEG615,ESEG616,ESEG617
0,11,Secretaria exclusiva,-,-,-,-,-,"secretaria de estado de segurança, defesa e ci...",Masculino,52,...,Sim,Não,Não,Não,Não,Sim,Não,Não,Não,Não
1,12,Secretaria em conjunto com outras políticas,Não,Não,Sim,Não,-,secretaria de justiça e seg. pública,Masculino,51,...,Sim,Não,Sim,Não,Não,Não,Não,Não,Não,Sim
2,13,Secretaria exclusiva,-,-,-,-,-,secretaria do estado de segurança pública,Masculino,62,...,Sim,Não,Sim,Sim,Sim,Sim,Sim,Sim,Sim,Não
3,14,Secretaria exclusiva,-,-,-,-,-,secretaria de estado de segurança publica,Masculino,50,...,-,-,Sim,Não,Não,Sim,Não,Não,Não,Não
4,15,Secretaria em conjunto com outras políticas,Não,Sim,Não,Não,-,secretaria de estado de segurança pública e de...,Masculino,39,...,Sim,Não,Sim,Não,Sim,Não,Sim,Não,Não,Não
5,16,Secretaria em conjunto com outras políticas,Não,Não,Sim,Sim,segurança publica,secretaria estadual de justiça e segurança pub...,Masculino,53,...,Não,Não,Não,Não,Não,Sim,Não,Não,Não,Não
6,17,Secretaria exclusiva,-,-,-,-,-,SECRETARIA DA SEGURANÇA PÚBLICA DO TOCANTINS,Masculino,46,...,Não,Não,Sim,Sim,Sim,Sim,Sim,Não,Não,Não
7,21,Secretaria exclusiva,-,-,-,-,-,secretaria de estado da segurança publica,Masculino,54,...,Sim,Não,Sim,Não,Não,Não,Não,Não,Não,Sim
8,22,Secretaria exclusiva,-,-,-,-,-,SECRETARIA ESTADUAL DE SEGURANÇA PÚBLICA DO PIAUI,Masculino,46,...,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Sim
9,23,Secretaria em conjunto com outras políticas,Não,Sim,Não,Não,-,SECRETARIA DAA SEGURSANÇA PUBLICA E DEFESA SOCIAL,Masculino,40,...,Não,Não,Sim,Sim,Não,Sim,Não,Não,Não,Não


In [4]:
df = df[['CodUF','ESEG03','ESEG04','ESEG05','ESEG06','ESEG07']]
df

,CodUF,ESEG03,ESEG04,ESEG05,ESEG06,ESEG07
0,11,"secretaria de estado de segurança, defesa e ci...",Masculino,52,Branca,Ensino superior completo
1,12,secretaria de justiça e seg. pública,Masculino,51,Branca,Mestrado
2,13,secretaria do estado de segurança pública,Masculino,62,Branca,Ensino superior completo
3,14,secretaria de estado de segurança publica,Masculino,50,Branca,Mestrado
4,15,secretaria de estado de segurança pública e de...,Masculino,39,Branca,Especialização
5,16,secretaria estadual de justiça e segurança pub...,Masculino,53,Branca,Ensino superior completo
6,17,SECRETARIA DA SEGURANÇA PÚBLICA DO TOCANTINS,Masculino,46,Branca,Mestrado
7,21,secretaria de estado da segurança publica,Masculino,54,Parda,Ensino superior completo
8,22,SECRETARIA ESTADUAL DE SEGURANÇA PÚBLICA DO PIAUI,Masculino,46,Preta,Especialização
9,23,SECRETARIA DAA SEGURSANÇA PUBLICA E DEFESA SOCIAL,Masculino,40,Branca,Especialização


In [5]:
cod_uf = pd.read_csv('MUNIC//MUNIC_educacao_2021.csv', sep=';')[['UF','Cod UF']]

In [6]:
x= cod_uf.pivot_table(columns=('UF','Cod UF'), aggfunc='size')


In [7]:
cod_uf = pd.DataFrame(x).reset_index()[['UF','Cod UF']]

In [8]:
df = df.merge(cod_uf, right_on='Cod UF',left_on='CodUF')
df

,CodUF,ESEG03,ESEG04,ESEG05,ESEG06,ESEG07,UF,Cod UF
0,11,"secretaria de estado de segurança, defesa e ci...",Masculino,52,Branca,Ensino superior completo,RO,11
1,12,secretaria de justiça e seg. pública,Masculino,51,Branca,Mestrado,AC,12
2,13,secretaria do estado de segurança pública,Masculino,62,Branca,Ensino superior completo,AM,13
3,14,secretaria de estado de segurança publica,Masculino,50,Branca,Mestrado,RR,14
4,15,secretaria de estado de segurança pública e de...,Masculino,39,Branca,Especialização,PA,15
5,16,secretaria estadual de justiça e segurança pub...,Masculino,53,Branca,Ensino superior completo,AP,16
6,17,SECRETARIA DA SEGURANÇA PÚBLICA DO TOCANTINS,Masculino,46,Branca,Mestrado,TO,17
7,21,secretaria de estado da segurança publica,Masculino,54,Parda,Ensino superior completo,MA,21
8,22,SECRETARIA ESTADUAL DE SEGURANÇA PÚBLICA DO PIAUI,Masculino,46,Preta,Especialização,PI,22
9,23,SECRETARIA DAA SEGURSANÇA PUBLICA E DEFESA SOCIAL,Masculino,40,Branca,Especialização,CE,23


In [9]:
df = df.drop(['CodUF'], axis=1)
df

,ESEG03,ESEG04,ESEG05,ESEG06,ESEG07,UF,Cod UF
0,"secretaria de estado de segurança, defesa e ci...",Masculino,52,Branca,Ensino superior completo,RO,11
1,secretaria de justiça e seg. pública,Masculino,51,Branca,Mestrado,AC,12
2,secretaria do estado de segurança pública,Masculino,62,Branca,Ensino superior completo,AM,13
3,secretaria de estado de segurança publica,Masculino,50,Branca,Mestrado,RR,14
4,secretaria de estado de segurança pública e de...,Masculino,39,Branca,Especialização,PA,15
5,secretaria estadual de justiça e segurança pub...,Masculino,53,Branca,Ensino superior completo,AP,16
6,SECRETARIA DA SEGURANÇA PÚBLICA DO TOCANTINS,Masculino,46,Branca,Mestrado,TO,17
7,secretaria de estado da segurança publica,Masculino,54,Parda,Ensino superior completo,MA,21
8,SECRETARIA ESTADUAL DE SEGURANÇA PÚBLICA DO PIAUI,Masculino,46,Preta,Especialização,PI,22
9,SECRETARIA DAA SEGURSANÇA PUBLICA E DEFESA SOCIAL,Masculino,40,Branca,Especialização,CE,23


In [10]:
df['ano']=2019

In [11]:
df = df.rename(columns={'UF':'sigla_uf',
                        'Cod UF':'cod_uf',
                        'ESEG03':'caracterizacao_orgao_gestor',
                        'ESEG04':'genero',
                        'ESEG05':'idade',
                        'ESEG06':'cor_raca',
                        'ESEG07':'grau_instrucao'})

In [12]:
df['caracterizacao_orgao_gestor']= df['caracterizacao_orgao_gestor'].str.title()

In [13]:
df = df[['ano','cod_uf','sigla_uf', 'caracterizacao_orgao_gestor','genero','idade','cor_raca','grau_instrucao']]

In [14]:
def transformar(nome):
    nome = re.sub(r"\bSeg. \b", "Segurança ", nome)
    nome = re.sub(r"\bSocail\b", "Social", nome)
    nome = re.sub(r"\Segursança\b", "Segurança", nome)
    nome = re.sub(r"\Daa\b", "Da", nome)
    return nome

In [15]:
df['caracterizacao_orgao_gestor']= df['caracterizacao_orgao_gestor'].apply(transformar) ##padronizando os nomes com erro de escrita

In [16]:
df['caracterizacao_orgao_gestor']=np.where(df['caracterizacao_orgao_gestor']=='Não Informou','Sem dados',df['caracterizacao_orgao_gestor'])
df['genero']=np.where(df['genero']=='Não informou','Sem dados',df['genero'])
df['cor_raca']=np.where(df['cor_raca']=='Não informou','Sem dados',df['cor_raca'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Não informou','Sem dados',df['grau_instrucao'])

df['idade']=np.where(df['idade']=='Não informou',np.nan,df['idade'])
df['idade'] =pd.to_numeric(df['idade'])


In [17]:
df['cor_raca'] = np.where(df['cor_raca']=='Pardo','Parda',df['cor_raca'])

In [18]:
limites = [0, 30, 50,65,100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [19]:
df= df[['ano', 'sigla_uf','cod_uf', 'caracterizacao_orgao_gestor','genero', 'faixa_etaria', 'cor_raca','grau_instrucao']]

In [20]:
# criando dicionário
dict_esco = {'Ensino superior completo':'Até Ensino Superior Completo',
             'Especialização':'Até Pós Graduação ou Mestrado',
             'Mestrado':'Até Pós Graduação ou Mestrado',
             'Doutorado':'Até Doutorado'}


In [21]:

df = df.replace({'grau_instrucao':dict_esco}) #substituindo valores para padronizar


In [24]:
df.columns

Index(['ano', 'sigla_uf', 'cod_uf', 'caracterizacao_orgao_gestor', 'genero',
       'faixa_etaria', 'cor_raca', 'grau_instrucao'],
      dtype='object')

In [30]:
df['ano'].unique()

array([2019], dtype=int64)

In [23]:
p

NameError: name 'p' is not defined

Subindo para o GBQ

In [ ]:
client = bigquery.Client()
dataset_ref = client.dataset('cargos_lideranca')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   ano                          27 non-null     int64   
 1   sigla_uf                     27 non-null     object  
 2   cod_uf                       27 non-null     int64   
 3   caracterizacao_orgao_gestor  27 non-null     object  
 4   genero                       27 non-null     object  
 5   faixa_etaria                 26 non-null     category
 6   cor_raca                     27 non-null     object  
 7   grau_instrucao               27 non-null     object  
dtypes: category(1), int64(2), object(5)
memory usage: 1.9+ KB


In [ ]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
        bigquery.SchemaField('sigla_uf','STRING',description='sigla da UF'),
        bigquery.SchemaField('cod_uf','INTEGER',description='Código do IBGE da UF'),
        bigquery.SchemaField('caracterizacao_orgao_gestor','STRING',description='Caracterização do órgão no qual o gestor está'),
        bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
        bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação'),
        bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
        bigquery.SchemaField('grau_instrucao','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação')

        ]      


In [ ]:
table_ref = dataset_ref.table('ESTADIC_perfil_gestor_seguranca_publica_tipo_orgao')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=e1d3074b-7d84-41b3-b82f-2c4a5ddfda8d>